In [ ]:
import os
import sys
import struct
import json
import uuid
import time
import random
import shutil
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
import base64
import unittest
import tempfile

In [ ]:
class SmartOTP:
    @staticmethod
    def generate_otp(x):
        """Generate 8-digit OTP based on 4-digit input X"""
        # Simulating a complex OTP generation mechanism
        random.seed(x)
        # Complex transformation to make output look random
        y = int(''.join([str(random.randint(0,9)) for _ in range(8)]))
        return y

In [ ]:
class MyFSMetadata:
    def __init__(self):
        self.computer_id = str(uuid.getnode())
        self.creation_time = time.time()
        self.files = {}
        self.system_password = None

In [ ]:
class MyFS:
    def __init__(self, volume_x_path, volume_y_path):
        self.volume_x_path = volume_x_path  # MyFS.Dat location
        self.volume_y_path = volume_y_path  # Metadata location
        self.max_files = 99
        self.max_file_size = 4 * 1024 * 1024 * 1024  # 4GB
        self.metadata = None

    def _generate_encryption_key(self, password):
        """Generate a secure encryption key from password"""
        salt = os.urandom(16)
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=salt,
            iterations=100000,
            backend=default_backend()
        )
        key = base64.urlsafe_b64encode(kdf.derive(password.encode()))
        # bytes arent json serializable, so we have to convert them to string for storage
        # when using them in Fernet, use base64.b64decode(key) to convert back to bytes
        return base64.b64encode(key).decode('utf-8'), base64.b64encode(salt).decode('utf-8')

    def _verify_computer(self):
        """Verify if current computer matches the one that created the file system"""
        if not self.metadata:
            raise ValueError("MyFS not initialized")
        return str(uuid.getnode()) == self.metadata.computer_id

    def create_myfs(self, system_password):
        """Create MyFS volume with initial metadata"""
        if os.path.exists(self.volume_x_path) or os.path.exists(self.volume_y_path):
            raise FileExistsError("MyFS volumes already exist")

        # Create empty files
        open(self.volume_x_path, 'wb').close()
        open(self.volume_y_path, 'wb').close()

        # Generate metadata
        metadata = MyFSMetadata()
        print(json.dumps(metadata.__dict__))
        key, salt = self._generate_encryption_key(system_password)
        metadata.system_password = {
            'salt': salt,
            'key': key
        }

        # Encrypt and save metadata
        fernet = Fernet(base64.b64decode(key))
        encrypted_metadata = fernet.encrypt(json.dumps(metadata.__dict__).encode())
        
        with open(self.volume_y_path, 'wb') as f:
            f.write(encrypted_metadata)

        self.metadata = metadata
        return True

    def change_system_password(self, old_password, new_password):
        """Change system-level password"""
        if not self._verify_computer():
            raise PermissionError("Unauthorized computer")

        # Decrypt existing metadata
        current_metadata = self._load_metadata(old_password)
        
        # Generate new encryption key
        new_key, new_salt = self._generate_encryption_key(new_password)
        current_metadata['system_password'] = {
            'salt': new_salt,
            'key': new_key
        }

        # Re-encrypt and save
        fernet = Fernet(new_key)
        encrypted_metadata = fernet.encrypt(json.dumps(current_metadata).encode())
        
        with open(self.volume_y_path, 'wb') as f:
            f.write(encrypted_metadata)

    def import_file(self, file_path, file_password=None):
        """Import a file into MyFS"""
        if not self._verify_computer():
            raise PermissionError("Unauthorized computer")

        if len(self.metadata.files) >= self.max_files:
            raise ValueError("MyFS volume is full")

        file_stats = os.stat(file_path)
        
        if file_stats.st_size > self.max_file_size:
            print(f"Warning: Large file {file_path} might have limited protection")

        # File metadata
        file_metadata = {
            'original_path': file_path,
            'import_time': time.time(),
            'size': file_stats.st_size,
            'has_password': bool(file_password)
        }

        # Encrypt file content if password provided
        with open(file_path, 'rb') as f:
            file_content = f.read()

        if file_password:
            file_key, file_salt = self._generate_encryption_key(file_password)
            fernet = Fernet(file_key)
            file_content = fernet.encrypt(file_content)
            file_metadata['file_salt'] = file_salt

        # Save encrypted file in MyFS.Dat
        with open(self.volume_x_path, 'ab') as f:
            file_offset = f.tell()
            f.write(file_content)

        # Update file metadata
        file_metadata['offset'] = file_offset
        self.metadata.files[os.path.basename(file_path)] = file_metadata

        return True

    def _detect_tampering(self):
        """Detect potential system tampering"""
        # Implement checks for unexpected modifications
        pass

    def check_dynamic_password(self):
        """Dynamic password verification mechanism"""
        x = random.randint(1000, 9999)  # 4-digit number
        print(f"Enter the matching 8-digit OTP for {x}")
        
        start_time = time.time()
        attempts = 0
        
        while attempts < 3:
            try:
                user_input = input("Enter OTP: ")
                elapsed_time = time.time() - start_time
                
                if elapsed_time > 20:
                    print("Time expired. Access denied.")
                    return False
                
                expected_otp = SmartOTP.generate_otp(x)
                if int(user_input) == expected_otp:
                    return True
                
                attempts += 1
                print(f"Incorrect OTP. {3 - attempts} attempts remaining.")
            
            except ValueError:
                print("Invalid input. Please enter a number.")
        
        return False


In [ ]:
# def main():
#     # Example usage
#     myfs = MyFS('X:/MyFS.Dat', 'Y:/MyFS_Metadata.dat')
    
#     # Initialize system
#     myfs.create_myfs('initial_system_password')
    
#     # Dynamic password check
#     if not myfs.check_dynamic_password():
#         print("System will now self-terminate.")
#         return
    
#     # Import a file
#     myfs.import_file('/path/to/important/file.txt', 'optional_file_password')

# if __name__ == "__main__":
#     main()

# __all__ = ['MyFS', 'SmartOTP']

In [ ]:
class TestMyFSFileSystem(unittest.TestCase):
    def setUp(self):
        """Set up test environment with temporary volumes"""
        self.temp_dir = tempfile.mkdtemp()
        self.volume_x_path = os.path.join(self.temp_dir, 'MyFS.Dat')
        self.volume_y_path = os.path.join(self.temp_dir, 'MyFS_Metadata.dat')
        self.myfs = MyFS(self.volume_x_path, self.volume_y_path)
        self.system_password = "test_password_123"

    def tearDown(self):
        """Clean up temporary files after each test"""
        if os.path.exists(self.volume_x_path):
            os.remove(self.volume_x_path)
        if os.path.exists(self.volume_y_path):
            os.remove(self.volume_y_path)

    def test_create_myfs(self):
        """Test MyFS volume creation"""
        result = self.myfs.create_myfs(self.system_password)
        self.assertTrue(result)
        self.assertTrue(os.path.exists(self.volume_x_path))
        self.assertTrue(os.path.exists(self.volume_y_path))

    def test_system_password_change(self):
        """Test changing system password"""
        # First create the MyFS
        self.myfs.create_myfs(self.system_password)
        
        # Change password
        new_password = 'new_test_password_456'
        self.myfs.change_system_password(self.system_password, new_password)
        
        # Verify password change (this would require additional method in MyFS)
        # For now, we'll just ensure no exception is raised

    def test_file_import_limits(self):
        """Test file import limitations"""
        self.myfs.create_myfs(self.system_password)
        
        # Create test files
        test_files = []
        for i in range(100):  # Try to exceed 99 file limit
            with tempfile.NamedTemporaryFile(delete=False, mode='w') as temp_file:
                temp_file.write(f'Test content {i}')
                test_files.append(temp_file.name)
        
        # Try importing files
        imported_count = 0
        for file_path in test_files:
            try:
                self.myfs.import_file(file_path)
                imported_count += 1
            except ValueError:
                break
        
        self.assertEqual(imported_count, 99)  # Should not exceed 99 files

    def test_large_file_import(self):
        """Test large file import warning"""
        self.myfs.create_myfs(self.system_password)
        
        # Create a large file (> 100MB)
        large_file_path = os.path.join(self.temp_dir, 'large_file.bin')
        with open(large_file_path, 'wb') as f:
            f.write(os.urandom(150 * 1024 * 1024))  # 150MB file
        
        # Import should work but with a warning
        try:
            result = self.myfs.import_file(large_file_path)
            self.assertTrue(result)
        except Exception as e:
            self.fail(f"Large file import failed: {e}")

    def test_dynamic_password_mechanism(self):
        """Test dynamic password verification mechanism"""
        # Mock the input mechanism for testing
        def mock_input(prompt):
            x = 1234  # predefined test input
            return str(SmartOTP.generate_otp(x))
        
        # Temporarily replace input
        original_input = __builtins__.get('input')
        __builtins__['input'] = mock_input
        
        try:
            result = self.myfs.check_dynamic_password()
            self.assertTrue(result)
        finally:
            # Restore original input
            __builtins__['input'] = original_input

    def test_file_encryption(self):
        """Test file import with optional encryption"""
        self.myfs.create_myfs(self.system_password)
        
        # Create a test file
        test_file_path = os.path.join(self.temp_dir, 'encrypted_test.txt')
        with open(test_file_path, 'w') as f:
            f.write('Sensitive information')
        
        # Import with file-level encryption
        file_password = 'file_specific_password'
        result = self.myfs.import_file(test_file_path, file_password)
        self.assertTrue(result)

    def test_metadata_preservation(self):
        """Test metadata preservation during import"""
        self.myfs.create_myfs(self.system_password)
        
        # Create a test file with known metadata
        test_file_path = os.path.join(self.temp_dir, 'metadata_test.txt')
        with open(test_file_path, 'w') as f:
            f.write('Metadata test content')
        
        # Import file
        self.myfs.import_file(test_file_path)
        
        # Check if metadata is preserved
        file_name = os.path.basename(test_file_path)
        file_metadata = self.myfs.metadata.files.get(file_name)
        
        self.assertIsNotNone(file_metadata)
        self.assertEqual(file_metadata['original_path'], test_file_path)
        self.assertIn('import_time', file_metadata)
        self.assertIn('size', file_metadata)

In [ ]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestMyFSFileSystem)
runner = unittest.TextTestRunner(verbosity=2)
result = runner.run(suite)

print("\nTest Summary:")
print(f"Total Tests: {result.testsRun}")
print(f"Passed: {result.testsRun - len(result.failures) - len(result.errors)}")
print(f"Failed: {len(result.failures)}")
print(f"Errors: {len(result.errors)}")

return result.wasSuccessful()